In [1]:
#!pip install tensorflow-data-validation

# 1. Import các thư viện cần thiết

In [2]:
import tensorflow as tf
import tensorflow_data_validation as tfdv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow_metadata.proto.v0 import schema_pb2

# 2. Load và xem thử dữ liệu

Bộ dữ liệu được sử dụng trong file notebook này là [Salary Prediction dataset](https://www.kaggle.com/datasets/rkiattisak/salaly-prediction-for-beginer)

In [3]:
# Đọc file dữ liệu
df = pd.read_csv('/content/drive/MyDrive/BD/salary.csv')
print(df)

      Age  Gender Education Level                      Job Title  \
0    32.0    Male      Bachelor's              Software Engineer   
1    28.0  Female        Master's                   Data Analyst   
2    45.0    Male             PhD                 Senior Manager   
3    36.0  Female      Bachelor's                Sales Associate   
4    52.0    Male        Master's                       Director   
..    ...     ...             ...                            ...   
370  35.0  Female      Bachelor's       Senior Marketing Analyst   
371  43.0    Male        Master's         Director of Operations   
372  29.0  Female      Bachelor's         Junior Project Manager   
373  34.0    Male      Bachelor's  Senior Operations Coordinator   
374  44.0  Female             PhD        Senior Business Analyst   

     Years of Experience    Salary  
0                    5.0   90000.0  
1                    3.0   65000.0  
2                   15.0  150000.0  
3                    7.0   60000.0 

In [4]:
# Chuyển các cột dữ liệu kiểu số về kiểu số nguyên
df['Age'] = np.floor(pd.to_numeric(df['Age'], errors='coerce')).astype('Int64')
df['Years of Experience'] = np.floor(pd.to_numeric(df['Years of Experience'], errors='coerce')).astype('Int64')
df['Salary'] = np.floor(pd.to_numeric(df['Salary'], errors='coerce')).astype('Int64')

In [5]:
# Chia dữ liệu thành 2 tập train và eval
train_df, eval_df = train_test_split(df, test_size=0.2, shuffle=False)

print('Train split size: ', train_df.shape)
print('Eval split size: ', eval_df.shape)

Train split size:  (300, 6)
Eval split size:  (75, 6)


# 3. Tạo ra số liệu thống kê

Kết quả được trả về ở bước này bao gồm:
- Đối với dữ liệu số:
  + Số lượng bản ghi dữ liệu.
  + % bản ghi bị thiếu.
  + Trung bình, độ lệch chuẩn, min, max.
  + % giá trị bằng không.
- Đối với dữ liệu phân loại:
  + Số lượng bản ghi dữ liệu.
  + % bản ghi bị thiếu.
  + Số lượng bản ghi unique.
  + Trung bình chiều dài chuỗi.

In [6]:
# Tạo ra số liệu thống kê của 2 tập dữ liệu train và eval
train_stats = tfdv.generate_statistics_from_dataframe(train_df)
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# 4. Tạo ra data schema

In [7]:
# Tạo ra schema từ số liệu thống kê đã có của tập train.
schema = tfdv.infer_schema(statistics=train_stats)

# Xem schema
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,optional,single,-
'Gender',STRING,optional,single,'Gender'
'Education Level',STRING,optional,single,'Education Level'
'Job Title',BYTES,optional,single,-
'Years of Experience',INT,optional,single,-
'Salary',INT,optional,single,-
'__index_level_0__',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male'"
'Education Level',"'Bachelor\'s', 'Master\'s', 'PhD'"


Domain là miền giá trị mong đợi của feature, ở đây của "Gender" là "Female" và "Male", của "Education Level" là "Bachelor's", "Master's" và "PhD" 

# 5. Tính toán và xuất ra anomalies của tập eval

In [8]:
# Thêm dòng vào tập eval để tạo ra anomalies
lengoctuong = {
    'Age': 21,
    'Gender': 'Ko xac dinh',
    'Education Level': 'BuS',
    'Job Title': 'SV',
    'Years of Experience': 69,
    'Salary': 690000
}

eval_df = eval_df.append(lengoctuong, ignore_index=True)
print(eval_df.tail(1))

    Age       Gender Education Level Job Title  Years of Experience  Salary
75   21  Ko xac dinh             BuS        SV                   69  690000


<ipython-input-8-b79ef5808d6f>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  eval_df = eval_df.append(lengoctuong, ignore_index=True)


In [9]:
# Tạo lại số liệu thống kê của tập eval sau khi thêm dòng mới
eval_stats = tfdv.generate_statistics_from_dataframe(eval_df)

# Tham chiếu đến schema mới được tạo ra để tìm lỗi trong tập eval
anomalies = tfdv.validate_statistics(statistics=eval_stats, schema=schema)

# Xem anomalies
tfdv.display_anomalies(anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'Gender',Unexpected string values,Examples contain values missing from the schema: Ko xac dinh (~1%).
'__index_level_0__',Column dropped,Column is completely missing
'Education Level',Unexpected string values,Examples contain values missing from the schema: BuS (~1%).


Ở đây anomalies chính là 2 giá trị "Ko xac dinh" của cột "Gender" và "BuS" của cột "Education Level" vì chúng không tồn tại trong tập train

# 6. Sửa anomalies trong schema

In [10]:
# Cách đầu tiên để sửa là có thể cho phép việc một loại giá trị có trong tập eval 
# nhưng ko có trong train bằng cách nới lỏng phần giá trị tối thiểu phải tới từ domain
edlevel_feature = tfdv.get_feature(schema, 'Education Level')
edlevel_feature.distribution_constraints.min_domain_mass = 0.9

# Hoặc có thể thêm một giá trị mới vào domain
gender_domain = tfdv.get_domain(schema, 'Gender')
gender_domain.value.append('Ko xac dinh')

# Ngoài ra có thể giới hạn range của một feature
tfdv.set_domain(schema, 'Age', schema_pb2.IntDomain(name='Age', min=17, max=90))

In [11]:
# Xem schema sau khi sửa, chú ý các feature 'Age', 'Gender'
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Age',INT,optional,single,min: 17; max: 90
'Gender',STRING,optional,single,'Gender'
'Education Level',STRING,optional,single,'Education Level'
'Job Title',BYTES,optional,single,-
'Years of Experience',INT,optional,single,-
'Salary',INT,optional,single,-
'__index_level_0__',INT,required,,-


,Values
Domain,
'Gender',"'Female', 'Male', 'Ko xac dinh'"
'Education Level',"'Bachelor\'s', 'Master\'s', 'PhD'"


In [12]:
# Xác thực lại số liệu thống kê của eval sau khi cập nhật schema 
updated_anomalies = tfdv.validate_statistics(eval_stats, schema)
tfdv.display_anomalies(updated_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'__index_level_0__',Column dropped,Column is completely missing


Có thể thấy đã không còn anomalies nào được tìm thấy nữa